# JCAPG Demo -- Jurix 2023

In [ ]:
!pip install openai

In [ ]:
import os
import openai
import json
from typing import Dict
from getpass import getpass

In [ ]:
# Enter OpenAI GPT-4 API key
OPEN_AI_API_KEY = getpass('Enter OpenAI API Key here: ')

In [ ]:
# Setup OpenAI GPT-4 API key
openai.api_key = OPEN_AI_API_KEY
os.environ['OPENAI_API_KEY'] = openai.api_key

In [ ]:
# This is the prompt used for Generating a pathway from scratch.
analysis_prompt:str = """
Your task is to extract the logical structure of a piece of legislation.
I will give you the text from a legislation. Determine the logical structure of the legislation and return it in a JSON format.

The structure to use contains two types of nodes: Question Nodes and Information Nodes. Question nodes correspond to a logical requirement, i.e. a question that needs to be answered. Information nodes correspond to a logical conclusion, i.e. a logical consequence of the answers to the questions. Each node has a unique key and a text. Question nodes additionally have follow-up nodes, relating to the consequence that arises from answering a question in a certain way. Here, the key of the follow-up question or information block should be provided.

In generating these nodes, you should follow these rules:
- Each question node should be a question that can be answered with a "yes" or a "no".
- Each information node should be a logical conclusion that can be drawn from the answers to the questions.
- Do not interpret the legislation "e contrario". If the legislation offers only a positive outcome, the negative outcome should contain words such as "according to this article", to show that there may be other ways to obtain the outcomne.
- Stick to the specific wording used in the provided legislation.
- Do not include any information that is not explicitly stated in the legislation.
- The pathway should not be recursive - make sure that there are no loops in the pathway. Each path should terminate in an info node with no follow-up node.
- Break up the logical sequence of a legal paragraph into as many question nodes as possible.
- Make sure that all logical requirements are included in the pathway.

Without any comment, provide only a RFC8259 compliant JSON response following this format without deviation.
{
  "1": {
    "question": "Your first question goes here",
    "yes_followup": "The key to another question if 'YES', an Information Node",
    "no_followup": "The key to another question if 'NO' or an Information Node"
  },
  "2": {
    "info" : "The information based on the user's response goes here",
    "next_followup": null  # example of  info bloc
  }
}
"""


In [ ]:
class SemanticAnalysis:
    """
    Class that analyses legislation and creates `JCAPG JSON`.
    """
    def __init__(self, article: str, retries=3, model='gpt-4',
                 temperature=1, top_p=1, frequency_penalty=0, presence_penalty=0) -> None:
        self.system_context = analysis_prompt
        self.retries = retries

        # Model args
        self.model = model
        self.temperature = temperature
        self.top_p = top_p
        self.frequency_penalty = frequency_penalty
        self.presence_penalty = presence_penalty

        self.data = self.analyse_article(article)

    def analyse_article(self, article_text: str):
        """
        Asks GPT-4 model to return a pathway of a given article.
        """
        for _ in range(self.retries + 1):
            try:
                response = openai.ChatCompletion.create(
                    model=self.model,
                    messages=[
                        {"role": "system", "content": self.system_context},
                        {"role": "user", "content": article_text},
                    ],
                    temperature=self.temperature,
                    top_p=self.top_p,
                    frequency_penalty=self.frequency_penalty,
                    presence_penalty=self.presence_penalty
                )
                return self.get_dict_data(response["choices"][0]["message"]["content"])  # type: ignore
            except Exception as e:
                print(f"Failed to generate completion after {self.retries}: {e}")

    @staticmethod
    def get_dict_data(data_string: str) -> Dict[str, str] or Dict[str, None]:
        """
        Converts the data_string into a data_dictionary.
        Raises an exception if the conversion fails.
        """
        try:
            data_dictionary = json.loads(data_string)
            return data_dictionary
        except:
            raise Exception("Failed to decode data_string as JSON.")


In [ ]:
def main():
    print("=== WELCOME TO `JCAPG` ===")

    # Enter API Key below -- must have access to GPT-4
    openai.api_key = os.getenv("OPENAI_API_KEY")
    article_txt = input("Please insert legislation: \n")
    print("\nCalling API for Semantic analysis...")

    output = SemanticAnalysis(article=article_txt)


    print("\rAPI called!")
    print("Pathway JSON Generated: \n\n")
    print(output.data)

    # End of program
    print("\n\n=== End of Program ===")

In [ ]:
if __name__ == '__main__':
    main()